<center>
<img src="https://tensorflowkorea.files.wordpress.com/2020/12/4.-e18492e185a9e186abe1848ce185a1-e18480e185a9e186bce18487e185aee18492e185a1e18482e185b3e186ab-e18486e185a5e18489e185b5e186abe18485e185a5e18482e185b5e186bce18483e185b5e186b8e18485e185a5e.png?w=972" width="250" height="250"><br>
</center>


 - (https://bit.ly/hg-07-3)

### 07-3 신경망 모델 훈련

- 핵심키워드
    - 드롭아웃
    - 콜백
    - 조기종료
    - 인공 신경망 모델을 훈련하는 모범 사례와 필요한 도구들을 살펴보겠다. 이런 도구들을 다뤄보면서 텐서플로와 케라스 API에 더 익숙해지자.
    
지금까지 인공 신경망에 대해 배우고 텐서플로의 케라스 API를 사용해 직접 만들어 보았다. 1개 이상의 층을 추가하여 심층 신경망을 구성하고 다양한 고급 옵티마이저를 적용하는 방법도 알아 보았다.